In [3]:
import numpy as np
import pandas as pd

In [4]:
data_yf = pd.read_csv('data/raw_data_yf.csv')

In [5]:
data_yf.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,macd,macd_signal,...,kc_lag_4,kc_lag_12,vwap_lag_4,vwap_lag_12,eom_lag_4,eom_lag_12,vpt_lag_4,vpt_lag_12,gain_loss_pct,win
0,2005-01-01,1.156786,1.243393,1.117857,1.236607,1.046911,5.402242e+09,AAPL,-13.373668,9.077173,...,339.237835,331.084166,345.780999,331.409048,-2.818720e+07,2.698811e+07,2.394464e+08,2.223275e+08,0.069002,1
1,2005-01-08,1.246964,1.328929,1.130357,1.253571,1.061272,1.119210e+10,AAPL,-36.761577,-0.090577,...,340.791002,330.940500,349.180141,330.474001,8.035444e+06,9.307251e+06,2.375267e+08,2.219597e+08,0.005299,1
2,2005-01-15,1.247321,1.278571,1.209821,1.258750,1.065657,3.584610e+09,AAPL,-54.666090,-11.005680,...,342.908001,331.149667,351.516383,328.027238,2.884406e+06,2.731130e+07,2.386018e+08,2.236732e+08,0.009163,1
3,2005-01-22,1.267500,1.321071,1.259821,1.321071,1.118418,3.848197e+09,AAPL,-68.065895,-22.417723,...,345.486002,331.484000,354.328406,327.959388,3.042372e+06,4.997345e+07,2.388381e+08,2.319839e+08,0.042265,1
4,2005-01-29,1.331786,1.426964,1.330536,1.407857,1.191891,4.674762e+09,AAPL,-77.781714,-33.490521,...,329.660633,332.883168,83.529489,330.784544,-3.794730e+09,8.434586e+07,-5.145638e+09,2.381677e+08,0.057119,1


In [6]:
data_yf.isna().any().any()

False

In [7]:
#create a list of dataframes
symbols = list(data_yf['Ticker'].value_counts().index)

df_list = []

# Loop over each unique ticker
for ticker in range(len(symbols)):
    df = data_yf[data_yf['Ticker'] == symbols[ticker]]
    
    # Rename all columns in df to "original_column_nameTicker"
    df.columns = [str(col) + str(symbols[ticker]) for col in df.columns]
    
    df = df.reset_index().drop(columns=['index'])#[0:990]
    
    # Append modified df to the list
    df_list.append(df)

In [8]:
num_null = 0

for df in df_list:
    if df.isna().any().any() == True:
        num_null +=1 
        
data = pd.concat(df_list, axis=1)


data.set_index('DateFITB', inplace=True)

In [9]:
date_values = [col for col in data.columns if 'Date' in col]
data.drop(columns=date_values, inplace=True)

Now, we will do Predict then Optimize -- Shared Learning

<h2> Predict then Optimize -- Shared Learning</h2>

In [10]:
#keep only float columns for model
float_columns = data.select_dtypes(include=['float64']).columns

data = data[float_columns]

data.dropna(inplace=True)

In [11]:
data.shape

(991, 18658)

In [12]:
data.head()

,OpenHUBB,HighHUBB,LowHUBB,CloseHUBB,Adj CloseHUBB,VolumeHUBB,macdHUBB,macd_signalHUBB,rsiHUBB,bb_highHUBB,...,kc_lag_4BIO,kc_lag_12BIO,vwap_lag_4BIO,vwap_lag_12BIO,eom_lag_4BIO,eom_lag_12BIO,vpt_lag_4BIO,vpt_lag_12BIO,gain_loss_pctBIO,winBIO
DateFITB,,,,,,,,,,,,,,,,,,,,,
2005-01-01,51.799999,51.840000,48.150002,48.580002,30.080427,727800.0,0.255113,-0.248084,42.114721,60.405934,...,90.797500,97.907333,85.792135,97.858890,2.332220e+07,-1.116786e+07,3.449147e+10,3.449041e+10,-0.037607,-1.0
2005-01-08,48.580002,50.610001,48.500000,50.570000,31.312634,1242900.0,-0.005079,-0.199483,45.677702,60.306677,...,90.368667,97.201167,87.244430,95.545796,1.426996e+08,-9.531843e+05,3.449253e+10,3.449035e+10,-0.002126,0.0
2005-01-15,50.570000,50.950001,49.500000,49.810001,30.842054,767300.0,-0.269502,-0.213487,44.549890,59.945749,...,90.268167,96.288167,87.611151,92.791668,1.187428e+07,-1.718185e+07,3.449247e+10,3.449017e+10,0.026195,1.0
2005-01-22,49.709999,49.750000,47.450001,48.849998,30.247620,983900.0,-0.550182,-0.280826,43.102197,59.844305,...,90.281167,95.458333,88.094430,89.521370,7.366711e+05,2.307328e+07,3.449248e+10,3.449073e+10,-0.015717,-1.0
2005-01-29,49.000000,51.020000,48.950001,50.889999,31.510763,755100.0,-0.601082,-0.344877,47.040560,59.802791,...,88.215500,94.569667,88.075265,87.662264,-2.400362e+08,-9.610319e+05,3.449228e+10,3.449055e+10,0.014719,1.0


In [13]:
data.to_csv('data/all_stock_data_transformed_horizontally.csv')

In [14]:
import tensorflow as tf
from scipy.optimize import minimize

In [ ]:
# Use GPU if available
device_name = "GPU" if tf.config.experimental.list_physical_devices('GPU') else "cpu"
print("Using device:", device_name)

Using device: GPU


In [ ]:
# Check type of GPU available on machine
physical_devices = tf.config.list_physical_devices()
print("Available devices:", physical_devices)

Available devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
# Prepare features and targets for model training
def prepare_features_targets(data_2):
#     indicators = calculate_technical_indicators(data)
#     returns = np.log(data / data.shift(1)).add_suffix('_returns')
#     full_data = pd.concat([indicators, returns], axis=1).dropna()
    features = data_2[[col for col in data_2.columns if 'gain_loss_' not in col]]
    targets = data_2[[col for col in data_2.columns if 'gain_loss_' in col]]
    return features, targets

# Build and compile a neural network
def build_model(input_shape, output_shape):
    with tf.device(device_name):
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu', kernel_initializer='he_normal',input_shape=(input_shape,)),
            tf.keras.layers.Dense(32, activation='relu',kernel_initializer='he_normal'),
            tf.keras.layers.Dense(output_shape)
        ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Portfolio optimization function with variance consideration and risk aversion
def optimize_portfolio(weights, model, features, historical_returns, risk_aversion=0.8):
    weights = weights / np.sum(weights)  # Normalize weights
    pred_returns = model.predict(np.array([features.iloc[-1]]))[0]
    expected_return = np.dot(weights, pred_returns)
    covariance_matrix = historical_returns.cov()
    portfolio_variance = np.dot(weights.T, np.dot(covariance_matrix, weights))
    # Objective function with risk aversion parameter
    return -expected_return + risk_aversion * portfolio_variance  # Maximize returns and penalize variance


In [ ]:
#set index to datetime
data.index = pd.to_datetime(data.index)

In [ ]:
# Main script
#features, targets = prepare_features_targets(data)
with tf.device(device_name):
    print(device_name)
    # Your model code here
    split_date = pd.Timestamp('2023-01-01')
    # Prepare your features and targets first
    features, targets = prepare_features_targets(data)
    # Split the data based on the threshold date
    train_features = features[features.index < split_date]
    test_features = features[features.index >= split_date]
    train_targets = targets[targets.index < split_date]
    test_targets = targets[targets.index >= split_date]
    model = build_model(train_features.shape[1], train_targets.shape[1])
    model.fit(train_features, train_targets, epochs=20, batch_size=5)

constraints = {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}
initial_weights = np.ones(len(symbols)) / len(symbols) # Equal weights
bounds = tuple((0, 1) for _ in symbols)


with tf.device(device_name):
    # Optimizing the portfolio on training data
    train_optimized_result = minimize(optimize_portfolio, initial_weights, args=(model, train_features, train_targets, 1), bounds=bounds, constraints=constraints)
    optimized_weights_train = train_optimized_result.x

    # Optimizing the portfolio on testing data
    test_optimized_result = minimize(optimize_portfolio, initial_weights, args=(model, test_features, test_targets, 1), bounds=bounds, constraints=constraints)
    optimized_weights_test = test_optimized_result.x

    # Calculating ROI
    train_predicted_returns = model.predict(train_features)
    train_roi = np.dot(optimized_weights_train, train_predicted_returns.mean(axis=0)) * 10000  # Assuming $10,000 initial investment

    test_predicted_returns = model.predict(test_features)
    test_roi = np.dot(optimized_weights_test, test_predicted_returns.mean(axis=0)) * 10000  # Assuming $10,000 initial investment


GPU
Epoch 1/20
188/188 [==============================] - 2s 7ms/step - loss: 14.4151
Epoch 2/20
188/188 [==============================] - 1s 6ms/step - loss: 7.3814
Epoch 3/20
188/188 [==============================] - 1s 7ms/step - loss: 4.2198
Epoch 4/20
188/188 [==============================] - 1s 6ms/step - loss: 3.1801
Epoch 5/20
188/188 [==============================] - 1s 6ms/step - loss: 2.7960
Epoch 6/20
188/188 [==============================] - 1s 6ms/step - loss: 2.3054
Epoch 7/20
188/188 [==============================] - 1s 6ms/step - loss: 2.2850
Epoch 8/20
188/188 [==============================] - 1s 6ms/step - loss: 1.9893
Epoch 9/20
188/188 [==============================] - 1s 6ms/step - loss: 1.9843
Epoch 10/20
188/188 [==============================] - 1s 6ms/step - loss: 1.8646
Epoch 11/20
188/188 [==============================] - 1s 6ms/step - loss: 1.9799
Epoch 12/20
188/188 [==============================] - 1s 6ms/step - loss: 1.9612
Epoch 13/20
188/188 

In [ ]:
print("Optimized Portfolio Weights (Train):", optimized_weights_train)
print("Training ROI:", train_roi)
print("Optimized Portfolio Weights (Test):", optimized_weights_test)
print("Testing ROI:", test_roi)

Optimized Portfolio Weights (Train): [1.16525522e-13 0.00000000e+00 2.75193332e-15 6.31702044e-14
 8.62284665e-14 0.00000000e+00 0.00000000e+00 2.24139006e-14
 7.62259658e-15 3.31744703e-13 0.00000000e+00 0.00000000e+00
 3.67957599e-14 1.14145869e-13 1.75657951e-15 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.21525275e-14 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.38522049e-14 0.00000000e+00 2.10689806e-14 5.91941296e-14
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 9.42976073e-14 1.01431914e-13 0.00000000e+00
 0.00000000e+00 1.73193329e-13 7.51757749e-15 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 4.43810312e-14 0.00000000e+00
 9.01615532e-02 2.66933872e-13 5.93838199e-13 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 4.12842513e-15 0.00000000e+00 0.

In [ ]:
tickers = [col.split('_')[0] for col in targets.columns]


def print_non_zero_weights(weights, title, tickers):
    # Using a threshold to consider weights effectively zero
    threshold = 1e-10
    non_zero_weights = {tickers[index]: weight for index, weight in enumerate(weights) if weight > threshold}
    print(f"{title} (non-zero weights):")
    for ticker, weight in non_zero_weights.items():
        print(f"{ticker}: Weight {weight}")

# Assuming 'symbols' is a list of ticker symbols corresponding to the indices of the weights! 
# Need to confirm this... otherwise the assumed tickers below will be wrong
print_non_zero_weights(optimized_weights_train, "Optimized Portfolio Weights (Train)", symbols)
print("Training ROI:", train_roi)
print_non_zero_weights(optimized_weights_test, "Optimized Portfolio Weights (Test)", symbols)
print("Testing ROI:", test_roi)



Optimized Portfolio Weights (Train) (non-zero weights):
HSY: Weight 0.09016155315258492
TECH: Weight 0.11745977820523382
AON: Weight 0.7923786686546628
Training ROI: 5191.205511434291
Optimized Portfolio Weights (Test) (non-zero weights):
NTAP: Weight 0.05049183020823118
MAS: Weight 0.1971562832200479
CI: Weight 0.04125584104801003
JNJ: Weight 0.590090266130837
FDX: Weight 0.12100577960049771
Testing ROI: 11733.204894699258


In [ ]:
# import numpy as np
# import pandas as pd
# import tensorflow as tf
# from scipy.optimize import minimize
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error

# # Prepare features and targets for model training
# def prepare_features_targets(data):
#     features = data[[col for col in data.columns if 'gain' not in col]]
#     targets = data[[col for col in data.columns if 'gain' in col]]
#     return features, targets

# # Build and compile a neural network
# def build_model(input_shape, output_shape):
#     model = tf.keras.Sequential([
#         tf.keras.layers.Dense(64, activation='relu', input_shape=(input_shape,)),
#         tf.keras.layers.Dense(32, activation='relu'),
#         tf.keras.layers.Dense(output_shape)
#     ])
#     model.compile(optimizer='adam', loss='mse')
#     return model

# # Portfolio optimization function with variance consideration and risk aversion
# def optimize_portfolio(weights, model, features, historical_returns, risk_aversion=0.8):
#     weights = weights / np.sum(weights)  # Normalize weights
#     pred_returns = model.predict(features)[-1]  # Predicting the last set of features
#     expected_return = np.dot(weights, pred_returns)
#     covariance_matrix = historical_returns.cov()
#     portfolio_variance = np.dot(weights.T, np.dot(covariance_matrix, weights))
#     return -expected_return + risk_aversion * portfolio_variance  # Maximize returns and penalize variance

# column_values = [col for col in data.columns if 'EXC' or 'APPL' in col]
# data_2 = data[column_values]

# symbols = ['EXC', 'AAPL']

# # Prepare features and targets
# features, targets = prepare_features_targets(data_2)

# # Split data into training and testing sets
# train_features, test_features, train_targets, test_targets = train_test_split(features, targets, test_size=0.2, random_state=42)

# # Build model
# model = build_model(train_features.shape[1], train_targets.shape[1])

# # Train model
# model.fit(train_features, train_targets, epochs=3, batch_size=32)

# # Optimizing portfolio using training data
# # train_returns = train_features.pct_change().dropna()
# initial_weights = np.ones(len(symbols)) / len(symbols)
# bounds = tuple((0, 1) for _ in symbols)
# constraints = {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}

# optimized_result_train = minimize(optimize_portfolio, initial_weights, args=(model, train_features, train_targets, 0.8), bounds=bounds, constraints=constraints)
# optimized_weights_train = optimized_result_train.x

# # Calculate ROI for the training set
# train_predicted_returns = model.predict(train_features)
# train_roi = np.dot(optimized_weights_train, train_predicted_returns.mean(axis=0)) * 10000  # Assuming $10,000 initial investment

# # Optimizing portfolio using testing data
# test_returns = test_features.pct_change().dropna()
# optimized_result_test = minimize(optimize_portfolio, initial_weights, args=(model, test_features, test_targets, 0.8), bounds=bounds, constraints=constraints)
# optimized_weights_test = optimized_result_test.x

# # Calculate ROI for the testing set
# test_predicted_returns = model.predict(test_features)
# test_roi = np.dot(optimized_weights_test, test_predicted_returns.mean(axis=0)) * 10000  # Assuming $10,000 initial investment

# print(f"Training ROI: ${train_roi:.2f}")
# print(f"Testing ROI: ${test_roi:.2f}")
# print("Optimized Weights for Training Set:", optimized_weights_train)
# print("Optimized Weights for Testing Set:", optimized_weights_test)


Non Shared Learning


In [ ]:
data_yf

In [ ]:
#Need a train loader and a test loader
#Need to convert to optDataset, then wrap with DataLoader

In [ ]:
!pip install PyPl gurobipy pyepo

Create a LSTM-based model for stock prediction tasks

In [ ]:
import torch.nn.functional as F
from torch import nn
import torch 
class StockLSTM(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers=1):
        super(StockLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim,num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        all_h, (h, c) = self.lstm(x)
        out = self.fc(all_h) # Apply Linear layer to outputs from all the hidden state.
        return out

In [295]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pyepo
from pyepo.model.grb import optGrbModel
import torch
from torch import nn
from torch.utils.data import DataLoader

m = 50 # change based on number of assets
cov = np.cov(np.random.randn(10, m), rowvar=False) # covariance matrix
optmodel = pyepo.model.grb.portfolioModel(m, cov) # build model

Auto-Sklearn cannot be imported.
Restricted license - for non-production use only - expires 2025-11-24


In [ ]:
import time

# train model
def trainModel(reg, loss_func, method_name, num_epochs=20, lr=1e-2):
    # set adam optimizer
    optimizer = torch.optim.Adam(reg.parameters(), lr=lr)
    # train mode
    reg.train()
    # init log
    train_loss_log = []
    loss_log_regret = [pyepo.metric.regret(reg, optmodel, loader_test)]
    # init elpased time
    elapsed = 0
    for epoch in range(num_epochs):
        # start timing
        tick = time.time()
        # load data
        train_loss = 0
        for i, data in enumerate(loader_train):
            x, c, w, z = data
            # cuda
            if torch.cuda.is_available():
                x, c, w, z = x.cuda(), c.cuda(), w.cuda(), z.cuda()
            # forward pass
            cp = reg(x)
            if method_name == "spo+":
                loss = loss_func(cp, c, w, z)
            elif method_name == "mse":
                loss = loss_func(cp, c)
            else:
                raise ValueError("Method name {} not supported".format(method_name))
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # record time
            tock = time.time()
            elapsed += tock - tick
            train_loss += loss.item()
        train_loss /= len(loader_train)
        train_loss_log.append(train_loss)
        regret = pyepo.metric.regret(reg, optmodel, loader_test)
        loss_log_regret.append(regret)
        print("Epoch {:2},  Loss: {:9.4f},  Regret: {:7.4f}%".format(epoch+1, train_loss, regret*100))
    print("Total Elapsed Time: {:.2f} Sec.".format(elapsed))
    return train_loss_log, loss_log_regret

Create a Predict-then-Optimize Model

In [ ]:

spop = pyepo.func.SPOPlus(optmodel, processes=1)

In [ ]:
# Hyperparameters
VOCAB_SIZE = #fill in once data is imported
EMBEDDING_DIM = #param we can optimize
HIDDEN_DIM = #also a param to optimize
learning_rate = #another optimizable param
epoch = #optimize
num_layers = #optimize
epochs = 20
learning_rate = 2e-3
method_name = "spo+"

# Instantiate the model
lstm = StockLSTM(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM,num_layers=num_layers)
loss_log_lstm_spo, loss_log_regret_lstm_spo = trainModel(lstm, loss_func=spop, method_name=method_name)
